In [ ]:
import networkx as nx
from node2vec import Node2Vec

import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

from collections import defaultdict

In [ ]:
file = open('train.txt', 'r')
train_data = []
for line in file:
    line = line.strip()
    line = line.split(' ')
    train_data.append(line)
totaledge = 0
for edge in train_data:
    for node in edge:
        totaledge = max(totaledge, int(node))
adj_matrix = np.zeros(shape = (totaledge + 1, totaledge + 1))

for row in train_data:
    for i in range(len(row)):
        for j in range(i + 1, len(row)):
            adj_matrix[int(row[i])][int(row[j])] += 1
            adj_matrix[int(row[j])][int(row[i])] += 1
            
rows, cols = np.where(adj_matrix > 0)
value = adj_matrix[rows, cols]
edges = list(zip(rows.tolist(), cols.tolist(), list(adj_matrix[rows, cols].astype(int))))

G = nx.Graph()
G.add_weighted_edges_from(edges)
    
neg_rows, neg_cols = np.where(adj_matrix == 0)
neg_edges = list(zip(neg_rows.tolist(), neg_cols.tolist(), list(adj_matrix[neg_rows, neg_cols].astype(int))))


In [ ]:
test_set = pd.read_csv('test-public.csv')
test_set = test_set.iloc[:, [1, 2]]
test_set = test_set.values.tolist()
prediction_shortest = []
for i in test_set:
    if i[0] in G and i[1] in G and nx.has_path(G, i[0], i[1]):
        prediction_shortest.append(nx.shortest_path_length(G, i[0], i[1], weight='weight'))
    else:
        prediction_shortest.append(9999)




In [ ]:
result = []
for i in range(len(test_set)):
    score = 1 / prediction_shortest[i]
    result.append([i + 1, score])
columnName = ['Id', 'Predicted']
output = pd.DataFrame(columns = columnName, data = result)
output.to_csv('output.csv', index = 0)
prediction_shortest

In [ ]:
## node2vec
node2vec = Node2Vec(G, dimensions = 64, walk_length = 10, num_walks = 10, workers = 4)
model = node2vec.fit(window = 10, min_count = 1, batch_words = 4)
model.wv.most_similar('2')

In [ ]:
model.wv.save_word2vec_format('embeddings.txt')
emb_file = open('embeddings.txt', 'r')
emb_dict = {}
emb_file.readline()
for line in emb_file:
    line = line.strip()
    line = line.split(' ')
    for i in range(1, len(line)):
        line[i] = float(line[i])
    emb_dict[line[0]] = line[1:]


In [ ]:
def cosineScore(x, y):
    result1 = 0
    result2 = 0
    result3 = 0
    for i in range(len(x)):
        result1 += x[i] * y[i]
        result2 += x[i] ** 2
        result3 += y[i] ** 2
    return result1 / ((result2 * result3) ** 0.5)

test_set = pd.read_csv('test-public.csv')
test_set = test_set.iloc[:, [1, 2]]
test_set = test_set.values.tolist()
result = []
i = 1
for pair in test_set:
    source = str(pair[0])
    sink = str(pair[1])
    if source in emb_dict.keys() and sink in emb_dict.keys():
        score = cosineScore(emb_dict[source], emb_dict[sink])
    else:
        score = 0
    result.append([i, score])
    i += 1
columnName = ['Id', 'Predicted']
output = pd.DataFrame(columns = columnName, data = result)
output.to_csv('output.csv', index = 0)

In [ ]:
x_train, y_train = [], []
all_links = edges + neg_edges
len(all_links)
len(neg_edges)

In [ ]:

import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from node2vec import Node2Vec
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report
import multiprocessing
from itertools import permutations

filepath = 'train.txt'

def read_text(filepath):
    author_edge = defaultdict(list)
    two_way = defaultdict(list)
    with open(filepath, 'r') as f:
        lines = f.readlines()
        for ln in lines:
            authors = ln.split()
            i = 0
            while i < len(authors):
                j = i + 1
                while j < len(authors):
                    if authors[i] != authors[j]:
                        if authors[j] not in author_edge[authors[i]]:
                            author_edge[authors[i]].append(authors[j])
                        if authors[j] not in two_way[authors[i]]:
                            two_way[authors[i]].append(authors[j])
                        if authors[i] not in two_way[authors[j]]:
                            two_way[authors[j]].append(authors[i])
                    j += 1
                i += 1

    return author_edge, two_way

def dict_to_list(author_dict):
    author_df = pd.DataFrame()
    author_1 = []
    author_2 = []

    for key, item in author_dict.items():
        for i in item:
            author_1.append(int(key))
            author_2.append(int(i))

    author_df['a1'] = author_1
    author_df['a2'] = author_2

    return author_df, author_1, author_2

def weighted_edge(filepath):
    a1 = []
    a2 = []
    edge = pd.DataFrame()
    with open(filepath, 'r') as f:
        lines = f.readlines()
        for ln in lines:
            authors = ln.split()
            for i in range(len(authors)):
                for j in range(i+1, len(authors)):
                    a1.append(authors[i])
                    a2.append(authors[j])
    edge['a1'] = a1
    edge['a2'] = a2

    return edge

wedge = weighted_edge(filepath)
print(wedge.shape)
author_dict,_ = read_text(filepath)
author_df, author1, author2 = dict_to_list(author_dict)
author_df['link'] = 1
print(author_df.shape)
author_list = author1 + author2
author_list = list(dict.fromkeys(author_list))
author_list.sort()
print(len(author_list))
print(author_list)
print(author_list[0], author_list[-1])
missing_node = list(set(range(author_list[-1]+1))-set(author_list))
print(len(missing_node))
author_graph = nx.convert_matrix.from_pandas_edgelist(author_df, 'a1', 'a2')
adj_matrix = nx.to_numpy_matrix(author_graph, nodelist=author_list)
print(adj_matrix.shape)

def get_unconnected_author(adj_matrix, author_list):
    unconnected = []
    i = 0
    while i < adj_matrix.shape[0]:
        j = 0
        while j < i:
            if adj_matrix.item((i, j)) == 0:
                unconnected.append((author_list[i], author_list[j]))

            j += 1
        i += 1
    return unconnected

unconnected_list = get_unconnected_author(adj_matrix, author_list)
print(len(unconnected_list))
a1_uc = [i[0] for i in unconnected_list]
a2_uc = [i[1] for i in unconnected_list]
uc_df = pd.DataFrame({'a1': a1_uc, 'a2': a2_uc})
uc_df['link'] = 0
author_graph.add_nodes_from(missing_node)
_,uc_select = train_test_split(uc_df, test_size=author_df.shape[0])
print(uc_select)
data = pd.concat([author_df, uc_select])
print(data)
author_graph = nx.convert_matrix.from_pandas_edgelist(wedge, 'a1', 'a2')
author_graph.add_nodes_from(missing_node)
n2v = Node2Vec(author_graph, p=10, q=0.1, workers=multiprocessing.cpu_count()-1)
n2v_model = n2v.fit(window=10, min_count=1)
x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i, j in zip(data['a1'], data['a2'])]
y = data['link'].to_list()
xtrain, xtest, ytrain, ytest = train_test_split(np.array(x), data['link'], test_size=0.3, stratify=data['link'])
mlp = MLPClassifier(hidden_layer_sizes=100, activation='relu', solver='adam')
mlp.fit(xtrain, ytrain)
predict_proba = mlp.predict_proba(xtest)[:,1]
print('Test set AUC: ', roc_auc_score(ytest, predict_proba))
test_file = 'test-public.csv'
test = pd.read_csv(test_file)
t_x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(test['Source'], test['Sink'])]
t_predictions = mlp.predict_proba(t_x)
print(t_predictions[:,1])
pred = t_predictions[:,1]
index = list(range(1, len(pred)+1))
pred_data = pd.DataFrame()
pred_data['id'] = index
pred_data['Predicted'] = pred
pred_data.to_csv('mlp.csv', index=False)